In [5]:
import pandas as pd
import numpy as np
import torch
import pyro
import pyro.distributions as dist
from palmerpenguins import load_penguins

In [2]:
penguins = load_penguins()
penguins.dropna(how='any', axis=0, inplace=True)

In [3]:
penguins

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007
...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009


### Pyro

In [52]:
adelie_mask = (penguins['species'] == 'Adelie')
adelie_mass_obs = torch.from_numpy(penguins.loc[adelie_mask, 'body_mass_g'].values)

In [59]:
def model_adelie_penguin_mass(obs=None):
    # priors over params.
    sigma = pyro.sample('sigma', dist.HalfNormal(100., 2000.))
    mu = pyro.sample('mu', dist.Normal(4000, 3000))
    
    return pyro.sample('mass', dist.Normal(mu, sigma), obs=obs)

In [41]:
from pyro.infer import Predictive, NUTS, MCMC

In [60]:
kernel = NUTS(model_adelie_penguin_mass)
mcmc = MCMC(kernel, num_samples=2000, warmup_steps=1000)
mcmc.run(obs=adelie_mass_obs)

Sample: 100%|██████████| 3000/3000 [00:43, 69.27it/s, step size=8.85e-01, acc. prob=0.916] 


In [64]:
mcmc.summary()


                mean       std    median      5.0%     95.0%     n_eff     r_hat
        mu   3705.95     35.15   3706.10   3649.91   3765.94   1695.28      1.00
     sigma    435.47     23.45    434.91    395.63    470.59   1698.30      1.00

Number of divergences: 0


In [62]:
samples_1 = mcmc.get_samples()

In [63]:
samples_1

{'mu': tensor([3744.3864, 3686.3726, 3705.4038,  ..., 3754.9237, 3623.4612,
         3663.0637], dtype=torch.float64),
 'sigma': tensor([400.8673, 463.9541, 402.6411,  ..., 487.2579, 414.0635, 437.1876],
        dtype=torch.float64)}